# Exploring ECCC datasets archived on CaSPAr

***
IMPORTANT:

I quickly created this tutorial starting from another one that used different datasets but similar tools.  Importantly, there very well could be some code and / or markdown in here that is obsolete or that relates to that other tutorial and NOT this one.

So be suspicious of anything that looks odd - it might be - and correct as necessary

***


This notebook provides examples of how to read and explore the archived Environment and Climate Change Canada (ECCC) datasets that are available through the Canadian Surface Prediction Archive (CaSPAr).

[CaSPAr](caspar-data.ca "Canadian Surface Prediction Archive") is an archive of numerical weather and hydrologic prediction and reanalysis products issued by Environment and Climate Change Canada. Additional information can be found in the CaSPAr [documentation](https://github.com/julemai/CaSPAr/wiki "CaSPAr Documentation")

All datasets used in this notebook were downloaded through CaSPAr's graphical user interface. Users must first register with CaSPAr, and the system also requires a Globus ID and application to transfer and download the data.  Full information is available [here](https://github.com/julemai/CaSPAr/wiki/How-to-get-started-and-download-your-first-data).

## Requirements

Data is disseminated through CaSPAr as netCDF.

It's relatively easy to work with netCDF files in Python using the third-party library [xarray](http://xarray.pydata.org/en/stable/index.html "xarray documentation"), but first, make sure xarray is installed in your Python environment. This can be done with [Conda](https://conda.io "Conda documentation").

Or alternatively with [pip](https://pypi.org/project/pip/ "Pip Documentation").

Once installed, xarray can be imported.

In [ ]:
import xarray as xr

This example will also demonstrate plotting functionality using third-party libraries, including [Matplotlib](https://matplotlib.org/ "Matplotlib documentation") and [Cartopy](https://scitools.org.uk/cartopy/docs/latest/ "Cartopy documentation").  If you haven't done so already, these libraries will also need to first be installed in your Python environment using the same methodology as described above for xarray.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
import cartopy.crs as ccrs
import cartopy.feature as cf

## Working with xarray

### Loading datasets

A netCDF file's content can be accessed with xarray (see [Requirements](#Requirements)).  

We can open data using the xarray method `load_dataset`, which is a wrapper around `open_dataset`. 

CaSPAr filenames generally follow the convention of `YYYYMMDDTT.nc` (e.g., 2017042812.nc) where YYYYMMDD is the date (e.g., 20170428 = April 28, 2017) and TT is the forecast issue time in UTC/Zulu (e.g., 12Z).  All deterministic products follow this convention.  Additionally, ensemble products follow a contention `YYYYMMDDHH_EEE.nc` where EEE is the ensemble member (e.g., '001', '002'...).

In [ ]:
# unformatted filename string 
fname = '{fpath}/{fcst_vdate}{fcst_vtime}.nc'

# parameters
fpath = 'data' # obtained from CaSPAr previously and put in this folder
fcst_vdate='20100919'  # forecast valid date
fcst_vtime='12' # forecast valid time is always 12 for Reanalysis

# formatted string
fname.format(fpath=fpath, fcst_vdate=fcst_vdate, fcst_vtime=fcst_vtime)

CaSPAr provides a separate netCDF file for each forecast issued at a given date/time, with each file containing whatever forecast variables and forecast horizon you choose. The choices vary by product/modelling system: for example, the Regional Determinisitic Reforecast System (RDRS) reanalysis is issued once per day at 12Z.

In [ ]:
ds = xr.load_dataset(fname.format(fpath=fpath, fcst_vdate=fcst_vdate, fcst_vtime=fcst_vtime))

Once loaded, xarray can display a summary of the file contents by calling the variable directly in interactive Python mode or by using a print() statement.

In [ ]:
# interactive
ds

From this summary you can see that there are only two dimensions in each CaPA grib file, the geographic coordinates `y` and `x`. 

In [ ]:
ds.dims

Each file contains 24 hours of data, but we can also open multiple files to create a longer time-series. If files are stored locally, it is suggested to use xarray's [open_mfdataset]("https://xarray.pydata.org/en/stable/generated/xarray.open_mfdataset.html") method.

In [ ]:
# parameters
fpath = 'data' # obtained from CaSPAr previously and put in this folder
#fcst_vdate=f'201009{day}'  # forecast valid date
fcst_vtime='12' # forecast valid time is always 12 for Reanalysis

paths = []


for day in range(19,23):
    print(day)
    fcst_vdate=f'201009{day}' 
    print(fcst_vdate)
    paths.append(fname.format(fpath=fpath, fcst_vdate=fcst_vdate, fcst_vtime=fcst_vtime))

print(paths)
ds = xr.open_mfdataset(paths)

In [ ]:
ds

### Available Variables 
The reanalysis has two precipitation variables: a modelled prediction and an analysis.  

The analysis is preferable as it is the post-processed version of the modelled prediction using CaPA 

In [ ]:
# compute rolling means 
ds['6h-precip'] = ds['RDRS_v2.1_A_PR0_SFC'].rolling(time=6).mean()
ds['12h-precip'] = ds['RDRS_v2.1_A_PR0_SFC'].rolling(time=12).mean()
ds['24h-precip'] = ds['RDRS_v2.1_A_PR0_SFC'].rolling(time=24).mean()
ds['36h-precip'] = ds['RDRS_v2.1_A_PR0_SFC'].rolling(time=36).mean()
ds['48h-precip'] = ds['RDRS_v2.1_A_PR0_SFC'].rolling(time=48).mean()
# compute the cumulative sum
ds['cumulative'] = ds['RDRS_v2.1_A_PR0_SFC'].cumsum(dim='time')
ds

Data in an xarray can be selected using any of several methods, applied to the dimensions of a dataset. 

Currenty, we have the following dimensions.

In [ ]:
ds.dims

So we can index from these in several different ways..  

In [ ]:
# by position, by integers
# e.g., select precip for time = 0, and include all x and y coordinates.
ds['RDRS_v2.1_A_PR0_SFC'][0,:,:]

## Plotting


### With xarray

In [ ]:
ds['RDRS_v2.1_A_PR0_SFC'][:,100,100].plot()

In [ ]:
ds['RDRS_v2.1_A_PR0_SFC'].isel(time=40).plot(x='rlon', y='rlat', cmap='GnBu')

### With Matplotlib

Since xarray uses matplotlib as its backend by default, you can combine the two and make use of the full functionality of matplotlib and also cartopy as well.

First, ensure the plots are displayed inline in this notebook, plus ignore some warnings from the Shapely and Cartopy libraries.

In [ ]:
%matplotlib inline

import warnings
from shapely.errors import ShapelyDeprecationWarning
from cartopy.io import DownloadWarning

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=DownloadWarning)

*** I TRIED BELOW TO CONVERT ROTATED LAT-LON to REGULAR using code that seemed to work for another dataset and location, but it keeps grabbing a point ouside of the grid longitude (i.e., too far west)

In [ ]:
# https://stackoverflow.com/a/64431809

import cartopy.crs as ccrs

rpole_lon = ds.rotated_pole[0].grid_north_pole_longitude
rpole_lat = ds.rotated_pole[0].grid_north_pole_latitude 
data_crs = ccrs.RotatedPole(rpole_lon, rpole_lat)

# Transform the point - src_crs is always Plate Carree for lat/lon grid
#x, y = data_crs.transform_point(-52.712, 47.5615, src_crs=ccrs.PlateCarree())
x, y = data_crs.transform_point(-50.712, 47.5615, src_crs=ccrs.PlateCarree())
stjohns = ds.sel(rlon=x, rlat=y, method='nearest')

print(x,y)
print(stjohns)
#print(stjohns.rlat)
stjohns["RDRS_v2.1_A_PR0_SFC"].plot()

In [ ]:
stjohns

In [ ]:
import hvplot.xarray

In [ ]:
ds["1h-precip_mm"]=ds["RDRS_v2.1_A_PR0_SFC"]*1000
ds["6h-precip_mm"]=ds["6h-precip"]*1000
ds["12h-precip_mm"]=ds["12h-precip"]*1000
ds["24h-precip_mm"]=ds["24h-precip"]*1000
ds["48h-precip_mm"]=ds["48h-precip"]*1000
ds["cumulative_mm"]=ds["cumulative"]*1000

Now we'll plot it on a global orthographic projection to see the full spatial domain of the product.

In [ ]:
import cartopy.crs as crs

proj = crs.Orthographic(-90, 30)

In [ ]:
# plot cumulative precip for one (the final) time step only, and mask values below 0.1 threshold otherwise seems to be some erroneous data (easier to see this later on in this notebook as well)
ds["cumulative_mm"].where(ds['cumulative_mm']>0.1).isel(time=len(ds.time)-1).hvplot.quadmesh('lon', 'lat', 
                                                      geo=True, project=True, rasterize=True, 
                                                      xlim=(-70, -51.2), ylim=(42.7, 55),
                                                      dynamic=False, coastline=True, 
                                                      title='Hurricane Igor - Sep 19-22, 2010 : Cumulative Precipitation (mm)', 
                                                      clabel='mm', 
                                                      frame_width=500)

In [ ]:
# we can change the colour scheme and increase the resolution of the coastline 
ds["cumulative_mm"].where(ds['cumulative_mm']>0.1).isel(time=len(ds.time)-1).hvplot.quadmesh('lon', 'lat', 
                                                      geo=True, project=True, rasterize=True, 
                                                      xlim=(-60, -52), ylim=(46.4, 52), 
                                                      dynamic=False, coastline='10m', # options are '10m/50m/110m'
                                                      title='Hurricane Igor - Sep 19-22, 2010 : Cumulative Precipitation (mm)',
                                                      clabel='mm', 
                                                      cmap='YlGnBu',
                                                      frame_width=500)

In [ ]:
# now plot 72 hours (time steps) to allow for a dynamic plot / animation
# make sure you set the clim (i.e., the limits of the colour bar) or else it will jump around
# we can also change the colour scheme and increase the resolution of the coastline 
ds["cumulative_mm"].where(ds['cumulative_mm']>0.1).isel(time=slice(0,73,1)).hvplot.quadmesh('lon', 'lat', 
                                                      geo=True, project=True, rasterize=True, 
                                                      xlim=(-60, -52), ylim=(46.4, 52), 
                                                      clim=(0,250), # set colour limit
                                                      dynamic=False, coastline='10m', # options are '10m/50m/110m'
                                                      # title='Hurricane Igor - Sep 19-22, 2010 : Cumulative Precipitation (mm)', # ** Removed title so that we can see date / time instead
                                                      clabel='mm', 
                                                      cmap='YlGnBu',
                                                      frame_width=500)

In [ ]:
# here we plot 1h-precip (in mm) instead of cumulative...

# in this case, plotting all data WITHOUT masking, you get zero values included and what look like some erroneous negative values for some reason.  
ds["1h-precip_mm"].isel(time=slice(5,10,1)).hvplot.quadmesh('lon', 'lat', 
                                                      geo=True, project=True, rasterize=True, 
                                                      xlim=(-70, -51.2), ylim=(42.7, 55), clim=(0,50), 
                                                      dynamic=False, coastline='50m', 
                                                      clabel='mm', 
                                                      cmap='cet_rainbow_bgyrm_35_85_c71',
                                                      frame_width=500)

In [ ]:
# mask 0 values and change colour scheme
ds['1h-precip_mm'].where(ds['1h-precip_mm'] > 0).isel(time=[44]).hvplot.quadmesh('lon', 'lat', 
                                                      geo=True, project=True, rasterize=True, 
                                                      xlim=(-70, -51.2), ylim=(42.7, 55), clim=(0,50), 
                                                      dynamic=False, coastline='50m', 
                                                      clabel='mm', 
                                                      cmap='cet_rainbow_bgyrm_35_85_c71',
                                                      frame_width=500)

In [ ]:
# still faulty values... mask > 0.1
ds['1h-precip_mm'].where(ds['1h-precip_mm'] > 0.1).isel(time=[44]).hvplot.quadmesh('lon', 'lat', 
                                                      geo=True, project=True, rasterize=True, 
                                                      xlim=(-70, -51.2), ylim=(42.7, 55), clim=(0,50), 
                                                      dynamic=False, coastline='50m', 
                                                      clabel='mm', 
                                                      cmap='cet_rainbow_bgyrm_35_85_c71',
                                                      frame_width=500)

In [ ]:
# animate!
ds['1h-precip_mm'].where(ds['1h-precip_mm'] > 0.1).isel(time=slice(0,73,1)).hvplot.quadmesh('lon', 'lat', 
                                                      geo=True, project=True, rasterize=True, 
                                                      xlim=(-70, -51.2), ylim=(42.7, 55), clim=(0,50), 
                                                      dynamic=False, coastline='50m', 
                                                      clabel='mm', 
                                                      cmap='cet_rainbow_bgyrm_35_85_c71',
                                                      frame_width=500)

## Plotting Wind Data

Had some success with wind data using the vectorfield plots from hvplot, but plenty of difficulty with some of the options for overlays (e.g., couldn't add "coastline = True") and to animate.  

So some of this may be helpful but needs work


In [ ]:
import numpy as np

In [ ]:
ds['mag'] = np.sqrt(ds['RDRS_v2.1_P_UUC_10m']**2 + ds['RDRS_v2.1_P_VVC_10m']**2)
ds['angle'] = (np.pi/2.) - np.arctan2(ds['RDRS_v2.1_P_UUC_10m']/ds['mag'], ds['RDRS_v2.1_P_VVC_10m']/ds['mag'])
ds

In [ ]:
import geoviews as gv

In [ ]:
# https://hvplot.holoviz.org/reference/xarray/vectorfield.html

# adding some options, tiles 
ds.isel(time=[44]).hvplot.vectorfield(x='lon', y='lat', angle='angle', mag='mag', 
                                      hover=False, geo=True, dynamic=True, #coastline=True,  # doesn't work
                                      project=True, xlim=(-70, -51.2), ylim=(42.7, 55),
                                      #cmap='cet_rainbow_bgyrm_35_85_c71',
                                      frame_width=600).opts(magnitude='mag') * gv.feature.coastline().opts(global_extent=False) # Works!!!
                                      # * gv.tile_sources.CartoLight  # Works but makes file larger

In [ ]:
# https://holoviews.org/reference/elements/bokeh/VectorField.html

# animations -- seem too slow to update and couldn't get fixed scaling of vectors to work

ds.isel(time=slice(68,73,1)).hvplot.vectorfield(x='lon', y='lat', angle='angle', mag='mag', 
                                      hover=False, geo=True, dynamic=True, rasterize=True, #coastline=True,
                                      project=True, xlim=(-60, -52), ylim=(46, 52), clim=(0,60),
                                      #cmap='cet_rainbow_bgyrm_35_85_c71',
                                      frame_width=600).opts(magnitude='mag', 
                                                            #magnitude=hv.dim('mag').norm()*0.2, rescale_lengths=False, # apparently needed for fixed scaling in animation, but didn't work
                                                            color='mag', cmap='viridis', 
                                                            pivot='tip') * gv.feature.coastline().opts(global_extent=False)
#* gv.tile_sources.CartoLight

In [ ]:
help(gv.tile_sources)

In [ ]:
# could try Holoviews example outright (instead of hvplot)

# https://holoviews.org/reference/elements/bokeh/VectorField.html